In [2]:
# Install required libraries
!pip install transformers==4.30.2 accelerate==0.21.0

# Import necessary libraries
import pandas as pd
import numpy as np
import re
import torch
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import gradio as gr

# Verify the installation
import transformers
import accelerate

print(transformers.__version__)
print(accelerate.__version__)

# Load data (assuming you have a dataframe `data` with symptom columns)
data = pd.read_csv('/content/Training_with_Treatments.csv')

# Initialize tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# Data Cleaning and Preprocessing
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'\W', ' ', text)  # Remove special characters
    return text.strip()

# Prepare input-output pairs
data['symptoms'] = data.drop(columns=['prognosis', 'Treatment']).apply(lambda row: ' '.join(row.astype(str)[row != 0].index), axis=1)

def prepare_data(data):
    # Prepare input text by concatenating the column names of non-zero values
    data['input_text'] = data.drop(columns=['prognosis', 'Treatment']).apply(lambda row: ' '.join([str(i) for i, val in enumerate(row) if val != 0]), axis=1) # Use enumerate to get indices of non-zero elements
    # Prepare output text by concatenating prognosis and treatment
    data['output_text'] = 'prognosis: ' + data['prognosis'] + ' treatment: ' + data['Treatment']
    return data

# Make a copy of data and apply prepare_data function
data = prepare_data(data.copy())

# Split data into training and validation sets
train_data, val_data = train_test_split(data, test_size=0.1, random_state=42)

# Convert to lists for easier handling
train_input_texts = train_data['input_text'].tolist()
train_output_texts = train_data['output_text'].tolist()
val_input_texts = val_data['input_text'].tolist()
val_output_texts = val_data['output_text'].tolist()

# Tokenize the training data
train_encodings = tokenizer(train_input_texts, truncation=True, padding=True, max_length=512)
train_decodings = tokenizer(train_output_texts, truncation=True, padding=True, max_length=512)

# Tokenize the validation data
val_encodings = tokenizer(val_input_texts, truncation=True, padding=True, max_length=512)
val_decodings = tokenizer(val_output_texts, truncation=True, padding=True, max_length=512)

class SymptomsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, decodings):
        self.encodings = encodings
        self.decodings = decodings

    def __getitem__(self, idx):
        # Get encodings for the given index and convert them to torch tensors
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Add the labels (decodings) to the item
        item['labels'] = torch.tensor(self.decodings['input_ids'][idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

# Create dataset objects for training and validation datasets
train_dataset = SymptomsDataset(train_encodings, train_decodings)
val_dataset = SymptomsDataset(val_encodings, val_decodings)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',              # Output directory for model predictions and checkpoints
    num_train_epochs=5,                  # Number of training epochs
    per_device_train_batch_size=8,       # Batch size for training
    per_device_eval_batch_size=8,        # Batch size for evaluation
    warmup_steps=500,                    # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,                   # Strength of weight decay
    logging_dir='./logs',                # Directory for storing logs
    logging_steps=10,                    # Log every 10 steps
    evaluation_strategy="epoch",         # Evaluate at the end of each epoch
    save_strategy="epoch",               # Save the model at the end of each epoch
    learning_rate=3e-4,                  # Learning rate
)

# Initialize the Trainer
trainer = Trainer(
    model=model,                          # The model to train
    args=training_args,                   # Training arguments
    train_dataset=train_dataset,          # Training dataset
    eval_dataset=val_dataset,             # Evaluation dataset
)

# Train the model
trainer.train()

# Function to map symptom names to binary vector
def symptoms_to_binary(symptoms):
    # Initialize binary vector with zeros
    binary_vector = np.zeros(len(data.columns) - 2)  # Exclude 'prognosis' and 'Treatment' columns

    # Split user input by space and map to binary vector
    for symptom in symptoms.split():
        if symptom in data.columns:
            col_idx = data.columns.get_loc(symptom)
            binary_vector[col_idx] = 1

    return binary_vector

# Function to predict disease and treatment based on symptoms
def predict_disease_and_treatment(symptoms):
    # Convert symptom names to binary vector
    binary_vector = symptoms_to_binary(symptoms)

    # Prepare input text for model
    input_text = 'symptoms: ' + ' '.join([str(i) for i, val in enumerate(binary_vector) if val != 0])

    # Tokenize input text
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Generate output
    with torch.no_grad():
        output_ids = model.generate(input_ids.to('cuda'), max_length=512, num_beams=4, early_stopping=True)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return output_text

# Function for Gradio interface
def gradio_predict(symptoms):
    prediction = predict_disease_and_treatment(symptoms)
    return prediction

# Create Gradio interface
interface = gr.Interface(
    fn=gradio_predict,                   # Function to be called for predictions
    inputs="text",                       # Input type is text
    outputs="text",                      # Output type is text
    title="Disease and Treatment Prediction",  # Title of the Gradio app
    description="Enter symptoms separated by spaces to get predicted disease and treatment."  # Description of the Gradio app
)

# Launch Gradio interface
interface.launch(share=True)             # Share=True will generate a shareable link for the interface


4.30.2
0.21.0


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be correc

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0846e66aeaa9fdc8ab.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
